# Data Preparation

In [2]:
import numpy as np
import pandas as pd
import plotly.offline as py
import plotly.graph_objects as go
import networkx as nx

In [3]:
nodes = pd.read_csv('./dataset/nodes.csv', dtype=str, encoding='utf-8')
nodes = nodes.dropna()
nodes["Type_num"] = nodes["Type_num"].astype('int')

edges = pd.read_csv('./dataset/edges.csv', dtype=str, encoding='utf-8')
edges = edges.dropna()
edges = edges.drop_duplicates(['Source','Target'])
edges['Weight'] = edges['Weight'].astype('int')

# Build The network Using NetworkX

In [4]:
G = nx.MultiDiGraph()

# add nodes
for i in range(len(nodes['Id'])):
    G.add_node(nodes['Id'][i])
    
# add edges
for i,j in edges.iterrows():
    G.add_edge(j["Source"],j["Target"], weight = j["Weight"], label = j["Label"])

# Define The Plotly Graph

## Graph1: Without Arrows 

### Kamada kawai layout layout

In [12]:
# kamada kawai layout
pos_kk = nx.kamada_kawai_layout(G)
for n, p in pos_kk.items():
    G.node[n]['pos_kk'] = p

In [13]:
edge_x = []
edge_y = []
for edge in G.edges:
    x0, y0 = G.nodes[edge[0]]['pos_kk']
    x1, y1 = G.nodes[edge[1]]['pos_kk']
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color="#888"),
    hoverinfo='none',
    mode='lines')

In [14]:
middle_hover_trace = go.Scatter(x=[], y=[], hovertext=[], mode='markers', hoverinfo="text",marker={'size': 15, 'color': '#d3d3d3'},opacity=0,line=None)
for edge in G.edges:
    x0, y0 = G.node[edge[0]]['pos_kk']
    x1, y1 = G.node[edge[1]]['pos_kk']
    hovertext = "Type: " + str(G.edges[edge]['label']) + "<br>" + "Weight: " + str(G.edges[edge]['weight'])
    middle_hover_trace['x'] += tuple([(x0 + x1) / 2])
    middle_hover_trace['y'] += tuple([(y0 + y1) / 2])
    middle_hover_trace['hovertext'] += tuple([hovertext])

In [15]:
node_x = []
node_y = []
for node in G.nodes:
    x, y = G.nodes[node]['pos_kk']
    node_x.append(x)
    node_y.append(y)
    
node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        reversescale=True,
        colorscale=[
        [0, "grey"],
        [1/7, "grey"],

        [1/7, "limegreen"],
        [2/7, "limegreen"],

        [2/7, "dodgerblue"],
        [3/7, "dodgerblue"],

        [3/7, "coral"],
        [4/7, "coral"],

        [4/7, "yellow"],
        [5/7, "yellow"],

        [5/7, "blueviolet"],
        [6/7, "blueviolet"],

        [6/7, "firebrick"],
        [7/7, "firebrick"]],
        colorbar=dict(
        title="Innovation Area",
        titleside="top",
        tickmode="array",
        tickvals=[1.4, 2.2, 3.1, 4, 4.8, 5.7, 6.5],
        ticktext=["Process", "Money", "People", "Structure", "Interfaces", "Strategy", "Infomation"],
        ticks="outside"),
        color=[],
        size=[],
        line=None))

In [16]:
node_trace.marker.color = nodes['Type_num']

node_predecessors = []
node_successors = []
node_size = []
for node in G.nodes:
    predecessors = [n for n in G.predecessors(node)]
    successors = [n for n in G.successors(node)]
    node_predecessors.append(len(predecessors))
    node_successors.append(len(successors))
    node_size.append(10*np.log(len(predecessors)+len(successors)+2.71828))
node_trace.marker.size = node_size

node_predecessors = [str(x) for x in node_predecessors]
node_successors = [str(x) for x in node_successors]
node_text = list(map('<br># of predecessors: '.join, zip(nodes['Label'], node_predecessors)))
node_text = list(map('<br># of successors: '.join, zip(node_text, node_successors)))
node_trace.text = node_text

In [17]:
fig = go.Figure(data=[edge_trace, node_trace, middle_hover_trace],
             layout=go.Layout(
                title='<br>Management Innovations Network',
                titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )

fig.show()

### Spring layout

In [11]:
# spring layout
pos_spring = nx.spring_layout(G)
for n, p in pos_spring.items():
    G.node[n]['pos_spring'] = p

In [12]:
edge_x = []
edge_y = []
for edge in G.edges:
    x0, y0 = G.nodes[edge[0]]['pos_spring']
    x1, y1 = G.nodes[edge[1]]['pos_spring']
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color="#888"),
    hoverinfo='none',
    mode='lines')

In [13]:
middle_hover_trace = go.Scatter(x=[], y=[], hovertext=[], mode='markers', hoverinfo="text",marker={'size': 15, 'color': '#d3d3d3'},opacity=0,line=None)
for edge in G.edges:
    x0, y0 = G.node[edge[0]]['pos_spring']
    x1, y1 = G.node[edge[1]]['pos_spring']
    hovertext = "Type: " + str(G.edges[edge]['label']) + "<br>" + "Weight: " + str(G.edges[edge]['weight'])
    middle_hover_trace['x'] += tuple([(x0 + x1) / 2])
    middle_hover_trace['y'] += tuple([(y0 + y1) / 2])
    middle_hover_trace['hovertext'] += tuple([hovertext])

In [14]:
node_x = []
node_y = []
for node in G.nodes:
    x, y = G.nodes[node]['pos_spring']
    node_x.append(x)
    node_y.append(y)
    
node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        reversescale=True,
        colorscale=[
        [0, "grey"],
        [1/7, "grey"],

        [1/7, "limegreen"],
        [2/7, "limegreen"],

        [2/7, "dodgerblue"],
        [3/7, "dodgerblue"],

        [3/7, "coral"],
        [4/7, "coral"],

        [4/7, "yellow"],
        [5/7, "yellow"],

        [5/7, "blueviolet"],
        [6/7, "blueviolet"],

        [6/7, "firebrick"],
        [7/7, "firebrick"]],
        colorbar=dict(
        title="Innovation Area",
        titleside="top",
        tickmode="array",
        tickvals=[1.4, 2.2, 3.1, 4, 4.8, 5.7, 6.5],
        ticktext=["Process", "Money", "People", "Structure", "Interfaces", "Strategy", "Infomation"],
        ticks="outside"),
        color=[],
        size=[],
        line=None))

In [15]:
node_trace.marker.color = nodes['Type_num']

node_predecessors = []
node_successors = []
node_size = []
for node in G.nodes:
    predecessors = [n for n in G.predecessors(node)]
    successors = [n for n in G.successors(node)]
    node_predecessors.append(len(predecessors))
    node_successors.append(len(successors))
    node_size.append(10*np.log(len(predecessors)+len(successors)+2.71828))
node_trace.marker.size = node_size

node_predecessors = [str(x) for x in node_predecessors]
node_successors = [str(x) for x in node_successors]
node_text = list(map('<br># of predecessors: '.join, zip(nodes['Label'], node_predecessors)))
node_text = list(map('<br># of successors: '.join, zip(node_text, node_successors)))
node_trace.text = node_text

In [16]:
fig = go.Figure(data=[edge_trace, node_trace, middle_hover_trace],
             layout=go.Layout(
                title='<br>Management Innovations Network',
                titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )

fig.show()

### Circular layout

In [17]:
# circular_layout layout
pos_circular = nx.circular_layout(G)
for n, p in pos_circular.items():
    G.node[n]['pos_circular'] = p

In [18]:
edge_x = []
edge_y = []
for edge in G.edges:
    x0, y0 = G.nodes[edge[0]]['pos_circular']
    x1, y1 = G.nodes[edge[1]]['pos_circular']
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color="#888"),
    hoverinfo='none',
    mode='lines')

In [19]:
middle_hover_trace = go.Scatter(x=[], y=[], hovertext=[], mode='markers', hoverinfo="text",marker={'size': 15, 'color': '#d3d3d3'},opacity=0,line=None)
for edge in G.edges:
    x0, y0 = G.node[edge[0]]['pos_circular']
    x1, y1 = G.node[edge[1]]['pos_circular']
    hovertext = "Type: " + str(G.edges[edge]['label']) + "<br>" + "Weight: " + str(G.edges[edge]['weight'])
    middle_hover_trace['x'] += tuple([(x0 + x1) / 2])
    middle_hover_trace['y'] += tuple([(y0 + y1) / 2])
    middle_hover_trace['hovertext'] += tuple([hovertext])

In [20]:
node_x = []
node_y = []
for node in G.nodes:
    x, y = G.nodes[node]['pos_circular']
    node_x.append(x)
    node_y.append(y)
    
node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        reversescale=True,
        colorscale=[
        [0, "grey"],
        [1/7, "grey"],

        [1/7, "limegreen"],
        [2/7, "limegreen"],

        [2/7, "dodgerblue"],
        [3/7, "dodgerblue"],

        [3/7, "coral"],
        [4/7, "coral"],

        [4/7, "yellow"],
        [5/7, "yellow"],

        [5/7, "blueviolet"],
        [6/7, "blueviolet"],

        [6/7, "firebrick"],
        [7/7, "firebrick"]],
        colorbar=dict(
        title="Innovation Area",
        titleside="top",
        tickmode="array",
        tickvals=[1.4, 2.2, 3.1, 4, 4.8, 5.7, 6.5],
        ticktext=["Process", "Money", "People", "Structure", "Interfaces", "Strategy", "Infomation"],
        ticks="outside"),
        color=[],
        size=[],
        line=None))

In [21]:
node_trace.marker.color = nodes['Type_num']

node_predecessors = []
node_successors = []
node_size = []
for node in G.nodes:
    predecessors = [n for n in G.predecessors(node)]
    successors = [n for n in G.successors(node)]
    node_predecessors.append(len(predecessors))
    node_successors.append(len(successors))
    node_size.append(10*np.log(len(predecessors)+len(successors)+2.71828))
node_trace.marker.size = node_size

node_predecessors = [str(x) for x in node_predecessors]
node_successors = [str(x) for x in node_successors]
node_text = list(map('<br># of predecessors: '.join, zip(nodes['Label'], node_predecessors)))
node_text = list(map('<br># of successors: '.join, zip(node_text, node_successors)))
node_trace.text = node_text

In [22]:
fig = go.Figure(data=[edge_trace, node_trace, middle_hover_trace],
             layout=go.Layout(
                title='<br>Management Innovations Network',
                titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )

fig.show()

### Shell layout

In [23]:
process = nodes[nodes['Type']=="Process"]["Id"].tolist()
money = nodes[nodes['Type']=="Money"]["Id"].tolist()
people = nodes[nodes['Type']=="People"]["Id"].tolist()
structure = nodes[nodes['Type']=="Structure"]["Id"].tolist()
strategy = nodes[nodes['Type']=="Strategy"]["Id"].tolist()
interfaces = nodes[nodes['Type']=="Interfaces"]["Id"].tolist()
information = nodes[nodes['Type']=="Information"]["Id"].tolist()
shells = [process, money, people, structure, strategy, interfaces, information]

In [24]:
# shell layout
pos_shell = nx.shell_layout(G, shells)
for n, p in pos_shell.items():
    G.node[n]['pos_shell'] = p

In [25]:
edge_x = []
edge_y = []
for edge in G.edges:
    x0, y0 = G.nodes[edge[0]]['pos_shell']
    x1, y1 = G.nodes[edge[1]]['pos_shell']
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color="#888"),
    hoverinfo='none',
    mode='lines')

In [26]:
middle_hover_trace = go.Scatter(x=[], y=[], hovertext=[], mode='markers', hoverinfo="text",marker={'size': 15, 'color': '#d3d3d3'},opacity=0,line=None)
for edge in G.edges:
    x0, y0 = G.node[edge[0]]['pos_shell']
    x1, y1 = G.node[edge[1]]['pos_shell']
    hovertext = "Type: " + str(G.edges[edge]['label']) + "<br>" + "Weight: " + str(G.edges[edge]['weight'])
    middle_hover_trace['x'] += tuple([(x0 + x1) / 2])
    middle_hover_trace['y'] += tuple([(y0 + y1) / 2])
    middle_hover_trace['hovertext'] += tuple([hovertext])

In [27]:
node_x = []
node_y = []
for node in G.nodes:
    x, y = G.nodes[node]['pos_shell']
    node_x.append(x)
    node_y.append(y)
    
node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        reversescale=True,
        colorscale=[
        [0, "grey"],
        [1/7, "grey"],

        [1/7, "limegreen"],
        [2/7, "limegreen"],

        [2/7, "dodgerblue"],
        [3/7, "dodgerblue"],

        [3/7, "coral"],
        [4/7, "coral"],

        [4/7, "yellow"],
        [5/7, "yellow"],

        [5/7, "blueviolet"],
        [6/7, "blueviolet"],

        [6/7, "firebrick"],
        [7/7, "firebrick"]],
        colorbar=dict(
        title="Innovation Area",
        titleside="top",
        tickmode="array",
        tickvals=[1.4, 2.2, 3.1, 4, 4.8, 5.7, 6.5],
        ticktext=["Process", "Money", "People", "Structure", "Interfaces", "Strategy", "Infomation"],
        ticks="outside"),
        color=[],
        size=[],
        line=None))

In [28]:
node_trace.marker.color = nodes['Type_num']

node_predecessors = []
node_successors = []
node_size = []
for node in G.nodes:
    predecessors = [n for n in G.predecessors(node)]
    successors = [n for n in G.successors(node)]
    node_predecessors.append(len(predecessors))
    node_successors.append(len(successors))
    node_size.append(10*np.log(len(predecessors)+len(successors)+2.71828))
node_trace.marker.size = node_size

node_predecessors = [str(x) for x in node_predecessors]
node_successors = [str(x) for x in node_successors]
node_text = list(map('<br># of predecessors: '.join, zip(nodes['Label'], node_predecessors)))
node_text = list(map('<br># of successors: '.join, zip(node_text, node_successors)))
node_trace.text = node_text

In [29]:
fig = go.Figure(data=[edge_trace, node_trace, middle_hover_trace],
             layout=go.Layout(
                title='<br>Management Innovations Network',
                titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )

fig.show()

## Graph2: With Arrows

In [30]:
edge_x = []
edge_y = []
for edge in G.edges:
    x0, y0 = G.nodes[edge[0]]['pos_kk']
    x1, y1 = G.nodes[edge[1]]['pos_kk']
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color="#888"),
    hoverinfo='none',
    mode='lines')

In [31]:
middle_hover_trace = go.Scatter(x=[], y=[], hovertext=[], mode='markers', hoverinfo="text",marker={'size': 15, 'color': '#d3d3d3'},opacity=0,line=None)
for edge in G.edges:
    x0, y0 = G.node[edge[0]]['pos_kk']
    x1, y1 = G.node[edge[1]]['pos_kk']
    hovertext = "Type: " + str(G.edges[edge]['label']) + "<br>" + "Weight: " + str(G.edges[edge]['weight'])
    middle_hover_trace['x'] += tuple([(x0 + x1) / 2])
    middle_hover_trace['y'] += tuple([(y0 + y1) / 2])
    middle_hover_trace['hovertext'] += tuple([hovertext])

In [32]:
node_x = []
node_y = []
for node in G.nodes:
    x, y = G.nodes[node]['pos_kk']
    node_x.append(x)
    node_y.append(y)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        reversescale=True,
        colorscale=[
        [0, "grey"],
        [1/7, "grey"],

        [1/7, "limegreen"],
        [2/7, "limegreen"],

        [2/7, "dodgerblue"],
        [3/7, "dodgerblue"],

        [3/7, "coral"],
        [4/7, "coral"],

        [4/7, "yellow"],
        [5/7, "yellow"],

        [5/7, "blueviolet"],
        [6/7, "blueviolet"],

        [6/7, "firebrick"],
        [7/7, "firebrick"]],
        colorbar=dict(
        title="Innovation Area",
        titleside="top",
        tickmode="array",
        tickvals=[1.4, 2.2, 3.1, 4, 4.8, 5.7, 6.5],
        ticktext=["Process", "Money", "People", "Structure", "Interfaces", "Strategy", "Infomation"],
        ticks="outside"),
        color=[],
        size=[],
        line=None))

In [33]:
node_trace.marker.color = nodes['Type_num']

node_predecessors = []
node_successors = []
node_size = []
for node in G.nodes:
    predecessors = [n for n in G.predecessors(node)]
    successors = [n for n in G.successors(node)]
    node_predecessors.append(len(predecessors))
    node_successors.append(len(successors))
    node_size.append(10*np.log(len(predecessors)+len(successors)+2.71828))
node_trace.marker.size = node_size

node_predecessors = [str(x) for x in node_predecessors]
node_successors = [str(x) for x in node_successors]
node_text = list(map('<br># of predecessors: '.join, zip(nodes['Label'], node_predecessors)))
node_text = list(map('<br># of successors: '.join, zip(node_text, node_successors)))
node_trace.text = node_text

In [34]:
fig = go.Figure(data=[edge_trace, node_trace, middle_hover_trace],
             layout=go.Layout(
                title='<br>Management Innovations Network',
                titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    ax=(G.node[edge[0]]['pos_kk'][0] + G.node[edge[1]]['pos_kk'][0]) / 2,
                    ay=(G.node[edge[0]]['pos_kk'][1] + G.node[edge[1]]['pos_kk'][1]) / 2, axref='x', ayref='y',
                    x=(G.node[edge[1]]['pos_kk'][0] * 3 + G.node[edge[0]]['pos_kk'][0]) / 4,
                    y=(G.node[edge[1]]['pos_kk'][1] * 3 + G.node[edge[0]]['pos_kk'][1]) / 4, xref='x', yref='y',
                    showarrow=True,
                    arrowcolor='#888',
                    arrowhead=3,
                    arrowsize=1.5,
                    arrowwidth=0.5,
                    opacity=1) for edge in G.edges],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )

fig.show()

## Graph3: Edge Width By Weight

In [35]:
edge_trace = []
for edge in G.edges:
    x0, y0 = G.nodes[edge[0]]['pos_kk']
    x1, y1 = G.nodes[edge[1]]['pos_kk']
    trace = go.Scatter(x=tuple([x0, x1, None]), y=tuple([y0, y1, None]),
                       mode='lines',
                       line=dict(width=G.edges[edge]['weight'], color="#d3d3d3"),
                       line_shape='spline',
                       opacity=1)
    edge_trace.append(trace)

In [36]:
middle_hover_trace = go.Scatter(x=[], y=[], hovertext=[], mode='markers', hoverinfo="text",marker={'size': 15, 'color': '#d3d3d3'},opacity=0,line=None)
for edge in G.edges:
    x0, y0 = G.node[edge[0]]['pos_kk']
    x1, y1 = G.node[edge[1]]['pos_kk']
    hovertext = "Type: " + str(G.edges[edge]['label']) + "<br>" + "Weight: " + str(G.edges[edge]['weight'])
    middle_hover_trace['x'] += tuple([(x0 + x1) / 2])
    middle_hover_trace['y'] += tuple([(y0 + y1) / 2])
    middle_hover_trace['hovertext'] += tuple([hovertext])

In [37]:
node_x = []
node_y = []
for node in G.nodes:
    x, y = G.nodes[node]['pos_kk']
    node_x.append(x)
    node_y.append(y)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        reversescale=True,
        colorscale=[
        [0, "grey"],
        [1/7, "grey"],

        [1/7, "limegreen"],
        [2/7, "limegreen"],

        [2/7, "dodgerblue"],
        [3/7, "dodgerblue"],

        [3/7, "coral"],
        [4/7, "coral"],

        [4/7, "yellow"],
        [5/7, "yellow"],

        [5/7, "blueviolet"],
        [6/7, "blueviolet"],

        [6/7, "firebrick"],
        [7/7, "firebrick"]],
        colorbar=dict(
        title="Innovation Area",
        titleside="top",
        tickmode="array",
        tickvals=[1.4, 2.2, 3.1, 4, 4.8, 5.7, 6.5],
        ticktext=["Process", "Money", "People", "Structure", "Interfaces", "Strategy", "Infomation"],
        ticks="outside"),
        color=[],
        size=[],
        line=None))

In [38]:
node_trace.marker.color = nodes['Type_num']

node_predecessors = []
node_successors = []
node_size = []
for node in G.nodes:
    predecessors = [n for n in G.predecessors(node)]
    successors = [n for n in G.successors(node)]
    node_predecessors.append(len(predecessors))
    node_successors.append(len(successors))
    node_size.append(10*np.log(len(predecessors)+len(successors)+2.71828))
node_trace.marker.size = node_size

node_predecessors = [str(x) for x in node_predecessors]
node_successors = [str(x) for x in node_successors]
node_text = list(map('<br># of predecessors: '.join, zip(nodes['Label'], node_predecessors)))
node_text = list(map('<br># of successors: '.join, zip(node_text, node_successors)))
node_trace.text = node_text

In [39]:
layout = go.Layout(
                title='<br>Management Innovations Network',
                titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    ax=(G.node[edge[0]]['pos_kk'][0] + G.node[edge[1]]['pos_kk'][0]) / 2,
                    ay=(G.node[edge[0]]['pos_kk'][1] + G.node[edge[1]]['pos_kk'][1]) / 2, axref='x', ayref='y',
                    x=(G.node[edge[1]]['pos_kk'][0] * 3 + G.node[edge[0]]['pos_kk'][0]) / 4,
                    y=(G.node[edge[1]]['pos_kk'][1] * 3 + G.node[edge[0]]['pos_kk'][1]) / 4, xref='x', yref='y',
                    showarrow=True,
                    arrowcolor= '#b3b3b3',
                    arrowhead=3,
                    arrowsize=2,
                    arrowwidth=0.5,
                    opacity=1) for edge in G.edges],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
)

# Create figure
fig = go.Figure(layout = layout)

# Add edge hovertext
fig.add_trace(middle_hover_trace)

# Add all edge traces
for trace in edge_trace:
    fig.add_trace(trace)

# Add node trace
fig.add_trace(node_trace)

# Show figure
fig.show()

Node Color: type of innovation
Node Size: # of prece + # of successors

Edge Width: edge weight